# Input Boundary Conditions

Boundary condition files have the most insane file format of all the insane Delft3D-FLOW file formats! Therefore, making input boundary conditions warrant a separate notebook.
